In [11]:
import numpy as np
import Levenshtein
import heapq
import time

# Function to read the input file
def read_file(filename):
    sequences = []
    classes = []
    with open(filename, 'r') as file:
        for line in file:
            parts = line.strip().split("\t")
            if len(parts) == 2:  # Ensure the line has exactly two parts
                seq, cls = parts
                sequences.append(seq)
                classes.append(cls)
            else:
                print(f"Skipping malformed line: {line.strip()}")
    return sequences, classes

# Create the distance matrix based on Levenshtein distance
def create_data_model(sequences):
    n = len(sequences)
    dist_matrix = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            if i != j:
                dist_matrix[i][j] = Levenshtein.distance(sequences[i], sequences[j])
    return dist_matrix

# A* heuristic: Minimum Spanning Tree (MST) cost of unvisited nodes
def mst_heuristic(remaining_nodes, dist_matrix):
    if len(remaining_nodes) <= 1:
        return 0

    # Prim's algorithm to calculate MST cost
    n = len(dist_matrix)
    visited = [False] * n
    pq = []
    
    # Start from the first unvisited node
    start_node = remaining_nodes[0]
    visited[start_node] = True
    total_cost = 0

    # Add edges from start_node to the priority queue
    for neighbor in remaining_nodes:
        if neighbor != start_node:
            heapq.heappush(pq, (dist_matrix[start_node][neighbor], neighbor))

    while pq:
        cost, node = heapq.heappop(pq)
        if not visited[node]:
            visited[node] = True
            total_cost += cost
            # Add edges from this node to other unvisited nodes
            for neighbor in remaining_nodes:
                if not visited[neighbor]:
                    heapq.heappush(pq, (dist_matrix[node][neighbor], neighbor))

    return total_cost

# A* TSP Solver
def tsp_a_star(dist_matrix):
    n = len(dist_matrix)
    
    # Priority queue for A* search (cost, current_node, visited_mask, path)
    pq = []
    heapq.heappush(pq, (0, 0, 1, [0]))  # Start from node 0 with cost 0

    # A dictionary to store the minimum cost to reach each state (node, visited_mask)
    best_cost = {}

    while pq:
        cost_so_far, current_node, visited_mask, path = heapq.heappop(pq)
        
        # If all nodes are visited, return to the start node
        if visited_mask == (1 << n) - 1:  # All nodes are visited
            return path + [0], cost_so_far + dist_matrix[current_node][0]

        # Check if we already found a cheaper way to reach this state
        if (current_node, visited_mask) in best_cost and best_cost[(current_node, visited_mask)] <= cost_so_far:
            continue
        best_cost[(current_node, visited_mask)] = cost_so_far

        # Expand the current node (go to unvisited neighbors)
        remaining_nodes = [i for i in range(n) if not (visited_mask & (1 << i))]
        for neighbor in remaining_nodes:
            next_mask = visited_mask | (1 << neighbor)
            new_cost = cost_so_far + dist_matrix[current_node][neighbor]
            
            # Calculate A* heuristic
            heuristic_cost = mst_heuristic(remaining_nodes, dist_matrix)
            estimated_total_cost = new_cost + heuristic_cost
            
            # Add the new state to the priority queue
            heapq.heappush(pq, (estimated_total_cost, neighbor, next_mask, path + [neighbor]))

    return [], float('inf')  # Return empty path and infinite cost if no solution is found

# Reorder the sequences based on the TSP path
def reorder_sequences(sequences, path):
    return [sequences[i] for i in path]

# Write the reordered sequences to an output file
def write_reordered_sequences(filename, reordered_sequences, classes):
    with open(filename, 'w') as file:
        for seq, cls in zip(reordered_sequences, classes):
            file.write(f"{seq} {cls}\n")

# Function to measure performance (execution time)
def measure_performance(input_file, output_file):
    # Track execution time
    start_time = time.time()

    sequences, classes = read_file(input_file)
    dist_matrix = create_data_model(sequences)

    optimal_path, min_cost = tsp_a_star(dist_matrix)
    if optimal_path:
        reordered_sequences = reorder_sequences(sequences, optimal_path)

        # Stop the timer
        end_time = time.time()

        # Write the reordered sequences to a file
        write_reordered_sequences(output_file, reordered_sequences, classes)

        # Print performance results
        print(f"Reordered sequences written to {output_file}")
        print(f"Execution time: {end_time - start_time:.4f} seconds")
        print(f"Minimum cost (TSP): {min_cost}")
    else:
        print("A* algorithm failed to find a valid solution.")

if __name__ == "__main__":
    input_file = "reducedhuman.txt"  # Input file containing sequences and classes
    output_file = "reordered_human_astar.txt"  # Output file for reordered sequences
    
    # Measure performance of the A* TSP solver
    measure_performance(input_file, output_file)


Reordered sequences written to reordered_human_astar.txt
Execution time: 236.9322 seconds
Minimum cost (TSP): 33207.0
